In [1]:
from collections import namedtuple
import os
os.chdir("../")

In [2]:
DataIngestionConfig = namedtuple("DataIngestionConfig",[
    "root_dir",
    "source_URL",
    "local_data_file",
    "unzip_dir"
])

In [ ]:
from dataclasses import dataclass
from pathlob import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_UR: str
    local_data_file: Path
    unzip_dir: Path


In [3]:
from DeepCNNClassifier.constants import *
from DeepCNNClassifier.utils import read_yaml,create_directories

In [4]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self):
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [5]:
import os 
import urllib.request as request
from zipfile import ZipFile

class DataIngestion:
    def __init__(self,config = DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
    
    def _get_updated_list_of_files(self, list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]

    def _preprocess(self, zf:ZipFile, f:str, working_dir:str):
        target_filepath = os.path.join(working_dir, f)
        if not os.path.exists(target_filepath):
            zf.extract(f, working_dir)

        if os.path.getsiz(target_filepath) == 0:
            os.remove(target_filepath)


    def unzip_and_clean(self):
        with ZipFile(file=self.config.local_data_file,mode="r") as zf:
            list_of_files = zf.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)
            for file in updated_list_of_files:
                self._preprocess(zf, file, self.config.unzip_dir)
                


In [7]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e

[2022-10-16 13:42:42,559: INFO: common]: yaml file: configs\config.yaml loaded successfully
[2022-10-16 13:42:42,561: INFO: common]: yaml file: params.yaml loaded successfully
[2022-10-16 13:42:42,566: INFO: common]: created directory at: artifacts
[2022-10-16 13:42:42,568: INFO: common]: created directory at: artifacts/data_ingestion


ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine